**Diplomado en Inteligencia Artificial y Aprendizaje Profundo**

# Modelo de clasificación para detección de fraude en tarjetas de crédito

##  Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Oleg Jarma, ojarmam@unal.edu.co
4. Maria del Pilar Montenegro, pmontenegro88@gmail.com

## Contenido

* [Introducción](#Introducción)
* [Importa módulos](#Importa-módulos)
* [Los datos](#Los-datos)
* [Lee los datos](#Lee-los-datos)
* [Funciones de activación](#Funciones-de-activación)
* [Regularizadores ](#Regularizadores)
* [El conjunto de datos cáncer de seno Wisconsin](#El-conjunto-de-datos-cáncer-de-seno-Wisconsin)
* [Lectura de datos](#Lectura-de-datos)
* [Preprocesamiento](#Preprocesamiento)
* [Analiza el desbalance en los targets](#Analiza-el-desbalance-en-los-targets)
* [Crea el modelo](#Crea-el-modelo)
* [Define métricas y compila](#Define-métricas-y-compila)
* [Entrena, tiene en cuenta los pesos de las transacciones](#Entrena-tiene-en-cuenta-los-pesos-de-las-transacciones)
* [Predicciones](#Predicciones)
* [Resultados-Calcula métricas](#Resultados-Calcula-métricas)
* [Evaluación del modelo](#Evaluación-del-modelo)
* [Conclusiones](#Conclusiones)

## Introducción

En esta lección hacemos ejemplo de clasificación para detectar fraude en el uso de trajeta de crédito. El ejemplo es adaptado de  [fchollet](https://github.com/keras-team/keras-io/blob/master/examples/structured_data/imbalanced_classification.py). Los datos están disponible en [Kaggle Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud/).

El propósito del ejemplo es ilustrar el caso de datos categóricos altamente desbalanceados. Un  problema muy común en la práctica. Usaremos varias métricas para evaluar el modelo.

La técnica se puede extender a la detección de datos anómalos en grades conjuntos de datos.

## Importa módulos

In [108]:
from __future__ import absolute_import, division, print_function, unicode_literals
#
#import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
#

from tensorflow.keras.models import Model
#
from tensorflow.keras.layers import Dense, Input, Activation, Dropout
#
from tensorflow.keras.utils import plot_model
#
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

#
from tensorflow.keras. metrics import FalseNegatives, FalsePositives, TrueNegatives
from tensorflow.keras. metrics import TruePositives, Precision, Recall
#
from tensorflow.keras.optimizers import Adam
#
from tensorflow.keras import callbacks
#
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
print(tf.__version__)

2.1.0


##  Los datos

Excepto por la transacción y el monto, no sabemos cuáles son las otras columnas (por razones de privacidad). Lo único que sabemos es que esas columnas que se desconocen ya se han escalado.

- *El monto de la transacción* es relativamente pequeño. La media de todas las monturas realizadas es de aproximadamente USD 88.

- *No hay valores faltantes*

- *La mayoría de las transacciones fueron no fraudulentas* (99,82%) del tiempo, mientras que las transacciones fraudulentas ocurren (018%) del tiempo en el marco de datos.

- *Transformación PCA*: La descripción de los datos dice que todas las características pasaron por una transformación PCA (técnica de reducción de dimensionalidad) (excepto por tiempo y cantidad).

- *Escalado*: tenga en cuenta que para implementar una transformación de PCA es necesario escalar previamente las características. (En este caso, todas las características de V se han escalado o al menos eso es lo que suponemos que hicieron las personas que desarrollan el conjunto de datos).

## Lee los datos

In [ ]:
fname = '../Datos/fraude/creditcard.csv'
training = pd.read_csv(fname)
training

## Preprocesamiento

### Construye tensores para entrada y salida

In [ ]:
# Separa features y targest
targets = training.pop('Class')
targets = np.array(targets, dtype = 'uint8')
targets.reshape((targets.shape[0],1))
#
features = np.array(training,dtype = 'float32' )
#
print("tamaño de los targets: ",targets.shape)
print("tamaño de los features: ",features.shape)

### Prepara datos de entrenamiento y de validación

In [ ]:
#
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

### Normaliza los datos

In [ ]:
# crea el objeto StandardScaler
scaler = StandardScaler()

# Ajusta los parámetros del scaler
scaler.fit(train_features)

# escala training y test
train_features = scaler.transform(train_features)
val_features = scaler.transform(val_features )


## Analiza el desbalance en los targets

In [ ]:
# Cuenta las frecuencias de los datos
counts = np.bincount(train_targets)
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

# Crea los pesos para el entrenamiento. Más peso a los menos frecuentes (1)
weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

In [ ]:
weight_for_1

## Crea el modelo

In [ ]:
inputs = Input(shape=(train_features.shape[1],),name='capa_entrada')

# vamos construyendo capa por capa
x = Activation('relu')(inputs)
x = Dense(256, activation='relu',name='primera_capa_oculta')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu',name='segunda_capa_oculta')(x)
x = Dropout(0.3)(x)
outputs = Dense(1, activation='sigmoid', name='capa_salida')(x)

# Creamos ahora el modelo
model = Model(inputs=inputs, outputs=outputs)


### Summary

In [ ]:
model.summary()
plot_model(model, to_file='../Imagenes/fraude_model.png', 
           show_shapes=True)

##  Define métricas y compila

- **accuracy_score**: En la clasificación con múltiples etiquetas, esta función calcula la precisión del subconjunto: el conjunto de etiquetas predichas para una muestra que coincide exactamente con el conjunto de etiquetas correspondiente en y_true.
- **precision_score**: es la razón $\frac{tp }{tp + fp}$ en donde $tp$ es el número de positivos verdadero y $fp$ el número de falsos positivos. El mejor valor es 1 y el peor valor es 0.
- **recall_score**:  es la relación $\frac{tp }{tp + fn}$ donde $tp$ es el número de verdaderos positivos y $fn$ el número de falsos negativos. El recuerdo es intuitivamente la capacidad del clasificador para encontrar todas las muestras positivas. El mejor valor es 1 y el peor valor es 0.

In [ ]:
# métricas
metrics = [
    FalseNegatives(name="fn"),
    FalsePositives(name="fp"),
    TrueNegatives(name="tn"),
    TruePositives(name="tp"),
    Precision(name="precision"),
    Recall(name="recall"),
]

model.compile(optimizer=Adam(1e-2), loss="binary_crossentropy", metrics=metrics)

## Entrena-tiene en cuenta los pesos de las transacciones

In [ ]:
callbacks = [callbacks.ModelCheckpoint("../Datos/fraude/fraud_model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

history = model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)

## Predicciones

In [80]:
# Predicting the Test set results
y_pred_val = model.predict(val_features)
y_pred_val[y_pred_val > 0.5] = 1
y_pred_val[y_pred_val <=0.5] = 0
y_pred_val.reshape((y_pred_val.shape[0]))
# predict the training set
y_pred_train = model.predict(train_features)
y_pred_train[y_pred_train > 0.5] = 1
y_pred_train[y_pred_train <=0.5] = 0
y_pred_train.reshape((y_pred_train.shape[0]))


array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

## Resultados-Calcula métricas

In [119]:
#
# elimina dimensiones sobrantes
val_targets = np.squeeze(val_targets)
y_pred_val = np.squeeze(y_pred_val)
train_targets = np.squeeze(train_targets)
y_pred_train = np.squeeze(y_pred_train)
#
# falsos negativos validación
fn_val = FalseNegatives()
fn_val.update_state(val_targets, y_pred_val)
fn_val = fn_val.result().numpy()
#
# falsos negativos entrenamiento
fn_train = FalseNegatives()
fn_train.update_state(train_targets, y_pred_train)
fn_train = fn_train.result().numpy()
# 
# falsos positivos validación
fp_val = FalsePositives()
fp_val.update_state(val_targets, y_pred_val)
fp_val = fp_val.result().numpy()
#
# falsos positivos entrenamiento
fp_train = FalsePositives()
fp_train.update_state(train_targets, y_pred_train)
fp_train = fp_train.result().numpy()
# 
# Precision validación
pre_val = Precision()
pre_val.update_state(val_targets, y_pred_val)
pre_val = pre_val.result().numpy()
#
# falsos negativos entrenamiento
pre_train = Precision()
pre_train.update_state(train_targets, y_pred_train)
pre_train = pre_train.result().numpy()
# 
# recall validación
re_val = Recall()
re_val.update_state(val_targets, y_pred_val)
re_val = re_val.result().numpy()
#
# recall entrenamiento
re_train = Recall()
re_train.update_state(train_targets, y_pred_train)
re_train = re_train.result().numpy()
# 

# diccionario
metricas = {'Falsos_positivos_train':fp_train, 'Falsos_positivos_val':fp_val,
           '%Falsos_positivos': np.round((fp_train+fp_val)/ len(training)*100,4),
           'Falsos_negativos_train':fn_train, 'Falsos_negativos_val':fn_val,
           '%Falsos_negativos': np.round((fn_train+fn_val)/ len(training)*100,4),
           'Precision_train': pre_train, 'Precision_val': pre_val,
           'Recall_train': re_train, 'Recall_val':re_val }

In [120]:
metricas

{'Falsos_positivos_train': 5318.0,
 'Falsos_positivos_val': 545.0,
 '%Falsos_positivos': 2.0586,
 'Falsos_negativos_train': 0.0,
 'Falsos_negativos_val': 11.0,
 '%Falsos_negativos': 0.0039,
 'Precision_train': 0.07271142,
 'Precision_val': 0.10509031,
 'Recall_train': 1.0,
 'Recall_val': 0.85333335}

## Evaluación del modelo

In [ ]:

def plot_metric(history, metric):
    train_metrics = history.history[metric]
    val_metrics = history.history['val_'+metric]
    epochs = range(1, len(train_metrics) + 1)
    plt.plot(epochs, train_metrics, 'bo--')
    plt.plot(epochs, val_metrics, 'ro-')
    plt.title('Entrenamiento y validación '+ metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(["train_"+metric, 'val_'+metric])
    plt.show()

In [ ]:
plot_metric(history, 'loss')

In [ ]:
plot_metric(history, 'accuracy')

## Conclusiones

Al final del entrenamiento, de 56,961 transacciones de validación, se obtuvo:

- Identificar correctamente a 66 de ellos como fraudulentos
- Faltan 9 transacciones fraudulentas
- A costa de marcar incorrectamente 441 transacciones legítimas

En el mundo real, se le daría un peso aún mayor a la clase 1,
para reflejar que los falsos negativos son más costosos que los falsos positivos.

**La próxima vez que su tarjeta de crédito sea rechazada en una compra en línea, esta es la razón.**

[Ir al inicio](#Contenido)